In [1]:
%run "../../PyUtils/common.ipynb"
import requests as rq;
from datetime import datetime;

patient_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_jlee", checkForDateTime=False);
patient_data[:]



<IPython.core.display.Javascript object>

,pname,gender,age,height,weight,pef_normal,pef_gy_bound,pef_yr_bound,climate_loc,pollution_loc
0,hand2,f,63,154,71,370,296,185,119,1
1,kimj,f,66,150,57,355,284,176,119,2
2,kuky,m,48,165,65,590,472,295,119,3
3,leeh2,f,61,159,63,385,308,192,112,4
4,kwonc,m,45,172,95,640,512,320,112,4
5,kimk,f,37,160,55,400,320,200,112,4
6,uhy,f,35,149,58,375,300,188,112,5


In [2]:
new = datetime.strptime("1/2/2015", '%m/%d/%Y')-datetime.strptime("1/1/2015", '%m/%d/%Y');
print(new.days!=1)

False


In [8]:
pef_data = pd.DataFrame();
pef_data[:]

for pn in patient_data.pname :
    if pn != 'kimk' :
        print(str(pn))
        timeCount = 0;
        dateCount = 0;
        pefCount = 0;
        pef_temp = pd.read_csv("./pef/" + str(pn) + "_PMPEF.csv", usecols=['Date','Time','PMPEF']);
        for i in range(1, len(pef_temp)) :
            if str(pef_temp.Time[i]) == 'nan' or pef_temp.Time[i] == " " :
                pef_temp.Time[i] = pef_temp.Time[i-1];
                timeCount = timeCount + 1;
            if pn == 'hand2' :
                datedelta = datetime.strptime(pef_temp.Date[i], '%m/%d/%y')-datetime.strptime(pef_temp.Date[i-1], '%m/%d/%y')
            else :
                datedelta = datetime.strptime(pef_temp.Date[i], '%m/%d/%Y')-datetime.strptime(pef_temp.Date[i-1], '%m/%d/%Y')
            if datedelta.days != 1 :
                dateCount = dateCount + 1;
                print(pef_temp.Date[i-1], "-", pef_temp.Date[i]);

            if str(pef_temp.PMPEF[i]) == 'nan' or pef_temp.PMPEF[i] == " " :
                pefCount = pefCount + 1;
                #print(pef_temp.Date[i-1], "-", pef_temp.Date[i]);

        print("timeMissing:", timeCount, ", dateMissing:", dateCount, ", pefMissing:", pefCount);
        pef_temp.info();
        pef_temp['pname'] = pd.Series(str(pn), index = pef_temp.index);
        pef_temp['ampm'] = pd.Series('pm', index = pef_temp.index);
        pef_data = pef_data.append(pef_temp, ignore_index = True);
        #pef_data.info();    

hand2
timeMissing: 33 , dateMissing: 0 , pefMissing: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 3 columns):
Date     708 non-null object
Time     708 non-null object
PMPEF    708 non-null int64
dtypes: int64(1), object(2)
memory usage: 16.7+ KB
kimj
timeMissing: 0 , dateMissing: 0 , pefMissing: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 3 columns):
Date     610 non-null object
Time     610 non-null object
PMPEF    610 non-null int64
dtypes: int64(1), object(2)
memory usage: 14.4+ KB
kuky
timeMissing: 0 , dateMissing: 0 , pefMissing: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 3 columns):
Date     642 non-null object
Time     642 non-null object
PMPEF    642 non-null int64
dtypes: int64(1), object(2)
memory usage: 15.1+ KB
leeh2
timeMissing: 214 , dateMissing: 0 , pefMissing: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entrie

In [9]:
#print(test)
pef_data.columns = [
    'date',
    'time',
    'pef',
    'pname',
    'ampm'];
pef_data[:]

,date,time,pef,pname,ampm
0,12/24/14,18:06,316,hand2,pm
1,12/25/14,18:31,319,hand2,pm
2,12/26/14,18:50,331,hand2,pm
3,12/27/14,18:58,321,hand2,pm
...,...,...,...,...,...
4001,11/18/2015,21:00,298,uhy,pm
4002,11/19/2015,21:00,263,uhy,pm
4003,11/20/2015,21:25,281,uhy,pm
4004,11/21/2015,21:25,235,uhy,pm


In [10]:
url = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=";

query  = "INSERT%20INTO%20pef_jlee (";
for col in range(0, len(pef_data.columns)) :
    query = query + str(pef_data.columns[col]);
    if col == len(pef_data.columns) - 1 :
        query = query + ")";
    else :
        query = query + ", ";
query = query + " VALUES (";
#print(query);
   
full_query = query;
for row in range(0, len(pef_data)) :
    full_query = query;
    for col in range(0, len(pef_data.columns)) :
        if pef_data.columns[col] == 'date' :
            full_query = full_query + "to_date('" + str(pef_data.loc[row][col]) + "', 'mm/dd/yy')";
        elif pef_data.columns[col] == 'time' :
            full_query = full_query + "'" + str(pef_data.loc[row][col]) + ":'";
        elif pef_data.columns[col] == 'pname' or pef_data.columns[col] == 'ampm' :
            full_query = full_query + "'" + str(pef_data.loc[row][col]) + "'";
        else :
            full_query = full_query + str(pef_data.loc[row][col]);
        
        if col == len(pef_data.columns) - 1 :
            full_query = full_query + ")"
        else :
            full_query = full_query + ", ";  
            
        full_query = full_query.replace(" ", "%20");
        full_query = full_query.replace(",", "%2C");
        full_query = full_query.replace("/", "%2F");
        full_query = full_query.replace("'", "%27");
        full_query = full_query.replace("(", "%28");
        full_query = full_query.replace(")", "%29");
     
    response = rq.get(url+full_query);
    #print(response.text[28:], row);

In [13]:
dfOriginal = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20date,time,pef%20FROM%20pef_jlee%20WHERE%20pname='uhy'%20and%20ampm='pm'%20ORDER%20BY%20date,%20time", checkForDateTime=False);
dfOriginal.columns = [
    'Date',
    'Time',
    'PMPEF'
];
dfOriginal[:]
#dfOriginal.to_csv('./newdata/uhy_prev_AMPEF.csv', sep = ',', index = False)

,Date,Time,PMPEF
0,2014-01-01,20:00:00,324
1,2014-01-02,20:00:00,324
2,2014-01-03,21:30:00,344
3,2014-01-04,21:30:00,327
...,...,...,...
686,2015-11-18,21:00:00,298
687,2015-11-19,21:00:00,263
688,2015-11-20,21:25:00,281
689,2015-11-21,21:25:00,235
